<a href="https://colab.research.google.com/github/agarwalsourabh55/Deep-learning/blob/NewsCategoryClassifier/NewsCategoryDeepLearning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [98]:
import pandas as pd
import torch
#a variable sourcefile is used to store the argument that is passed while running the script. Sys.argv is the argument taking the file name while executing the script and 1 indicates the argument number while 0 being the script that we are executing.
sourcefile = "/content/gdrive/MyDrive/News_Category_JSON.json"
data =  pd.read_json(sourcefile,lines= True)

In [54]:
data

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [58]:
## Lts See how many categories are present
len(data['category'].unique())

41

In [99]:
## Defining Tokenizer for the Embeddings
## For this use case trying hands on with bert tokenizer

!pip install transformers
from transformers import BertTokenizer
text = data['headline'].tolist()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer.encode_plus(text[0],
                                 add_special_tokens = True,
                                 truncation = True,
                                 padding = "max_length",
                                 return_attention_mask = True,
                                 return_tensors = "pt")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
import random
## labelencoding the target column
labels = dict(zip(data['category'].unique(), range(1,data['category'].nunique()+1)))
labels

{'ARTS': 31,
 'ARTS & CULTURE': 22,
 'BLACK VOICES': 7,
 'BUSINESS': 12,
 'COLLEGE': 20,
 'COMEDY': 9,
 'CRIME': 1,
 'CULTURE & ARTS': 41,
 'DIVORCE': 36,
 'EDUCATION': 19,
 'ENTERTAINMENT': 2,
 'ENVIRONMENT': 40,
 'FIFTY': 30,
 'FOOD & DRINK': 38,
 'GOOD NEWS': 28,
 'GREEN': 24,
 'HEALTHY LIVING': 26,
 'HOME & LIVING': 34,
 'IMPACT': 4,
 'LATINO VOICES': 18,
 'MEDIA': 14,
 'MONEY': 39,
 'PARENTING': 33,
 'PARENTS': 21,
 'POLITICS': 5,
 'QUEER VOICES': 10,
 'RELIGION': 16,
 'SCIENCE': 17,
 'SPORTS': 11,
 'STYLE': 23,
 'STYLE & BEAUTY': 35,
 'TASTE': 25,
 'TECH': 15,
 'THE WORLDPOST': 27,
 'TRAVEL': 13,
 'WEDDINGS': 37,
 'WEIRD NEWS': 6,
 'WELLNESS': 32,
 'WOMEN': 8,
 'WORLD NEWS': 3,
 'WORLDPOST': 29}

In [103]:
## Defining Dataloaders class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df,target_column):

        self.labels = [labels[label] for label in df[target_column]]
        self.texts = [tokenizer(text,
                                padding='max_length',
                                max_length = 512,
                                truncation=True,
                                return_tensors="pt") for text in df['headline']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [104]:
from sklearn.model_selection import train_test_split
# y= data['category']
# x = data.drop('category',axis=1)
train_data, val_data  = train_test_split(data,
                                         test_size = 0.3)

train, val = Dataset(train_data,'category'), Dataset(val_data,'category')

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=2,
                                               shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val,
                                             batch_size=2)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [88]:
train_data

,headline,authors,link,short_description,date
42160,Donald Trump Taps Tea Party Rep. Mike Pompeo T...,Christina Wilkie,https://www.huffingtonpost.com/entry/mike-pomp...,Pompeo shares many of Trump's concerns about M...,2016-11-18
172454,Kate Middleton Channels '70s Halston With New ...,Dana Oliver,https://www.huffingtonpost.com/entry/kate-midd...,...without getting all scissor happy.,2012-11-29
61151,America’s ‘Hamilton’ Obsession Is Officially S...,Kim Bellware,https://www.huffingtonpost.com/entry/hamilton-...,"It's all about the Hamiltons, baby.",2016-04-17
141777,How to Persuade Airline Employees to Give You ...,"Matt Gibson, Contributor\nCEO at Xpat Media an...",https://www.huffingtonpost.com/entry/how-to-pe...,"""How can I get upgraded?"" You may be wondering...",2013-10-20
43833,Australian PM Defends Right To Permanently Bar...,"Colin Packham, Reuters",https://www.huffingtonpost.com/entry/australia...,They are instead sent to detention camps on ne...,2016-10-31
...,...,...,...,...,...
121365,"Brains, Guns, and Preventable Murder","Charles J. Reid, Jr., ContributorProfessor of ...",https://www.huffingtonpost.com/entry/brains-gu...,The Isla Vista mass murder is proof that we ar...,2014-05-31
3558,Trevor Noah Unveils Terrifying Donald Trump-Th...,Lee Moran,https://www.huffingtonpost.com/entry/trevor-no...,"Trump may be ""the right man for the job"" to ta...",2018-03-21
131892,'Captain America: The Winter Soldier' Super Bo...,,https://www.huffingtonpost.com/entry/captain-a...,"Marvel released a new trailer for ""Captain Ame...",2014-02-02
161457,When Feeding Therapy Becomes Aversion Therapy,"Katja Rowell, M.D., Contributor\nAuthor and fa...",https://www.huffingtonpost.com/entry/when-feed...,Doctors and therapists must be better educated...,2013-03-26
